# Métodos multipasos


Se conoce como métodos multipasos a aquellos que emplean la aproximación en más de uno de los puntos de la red precedentes para determinar a aquellos que emplean la aproximación en el siguiente punto, a continuación se presentan en particular dos métodos.

**Método explícito de Adams - Bashfort de cuatro pasos**:


\begin{equation}
	\begin{split}
		w_0 & = \alpha, \;\;w_1 = \alpha_1, \;\;w_2 = \alpha_2, \;\;w_3 = \alpha_3, \\
		w_{i+1} & = w_i + \frac{h}{24} \left[55 f(t_{i}, w_{i}) - 59f(t_{i-1}, w_{i-1}) + 37f(t_{i-2}, w_{i-2}) - 9f(t_{i-3}, w_{i-3})\right],
	\end{split}
\end{equation}


para cada $i = 3, 4, 5, \dots, N - 1$, con un error de truncamiento dado por $\frac{251}{720}y^{(5)}(\mu_i)h^4$ para algún $\mu_i\in(t_{i-3},t_{i+1})$.

**Método implícito de Adams - Moulton de tres pasos**:


\begin{equation}
	\begin{split}
		w_0 & = \alpha, \;\;w_1 = \alpha_1,\;\; w_2 = \alpha_2\\
		w_{i+1} & = w_i + \frac{h}{24} \left[9 f(t_{i+1}, w_{i+1}) + 19 f(t_{i}, w_{i}) - 5 f(t_{i-1}, w_{i-1}) + f(t_{i-2}, w_{i-2})\right],
	\end{split}
\end{equation}


para cada $i = 3, 4, 5, \dots, N - 1$, con un error de truncamiento dado por $\frac{19}{720}y^{(5)}(\mu_i)h^4$ para algún $\mu_i\in(t_{i-2},t_{i+1})$.

In [1]:
# Importamos las librerias y funciones necesarias para replicar el método
import numpy as np
from numpy import exp

In [2]:
# Determinamos los parámetros donde trabajaremos
a = 0 # Punto inicial
b = 2 # Punto final
w0 = 2 # Condicion inicial
n = 10 # Número de pasos

In [3]:
# Definimos cadenas auxiliares para impresión de pantalla
punto = 'Punto'
aproxima = 'Aproximacion'
real = 'Real'
error = 'Error Absoluto'

In [4]:
# Definimos la función f(t,y)
def fty(t,y):
    fty = 2 * y + exp(2 * t)
    return fty

In [5]:
# Determinamos el tamaño de salto
h = (b - a) / n

In [6]:
# Determinamos los valores exactos y los error de aproximacion
# Primero definimos la solución real
def ftyR(t):
    ftyR = exp(2 * t) * (t + 2)
    return ftyR

In [7]:
# Generamos el arreglo de puntos y de aproximaciones donde trabajaremos
aprox = np.empty((2,n+1))

# La primer dimensión tendra los puntos donde trabajaremos, es decir, los puntos de la malla
aprox[0,:] = np.arange(a,b + h, h)

# Imprimimos la primer dimensión a fin de validar los resultados:
print(aprox[0,:])

[0.  0.2 0.4 0.6 0.8 1.  1.2 1.4 1.6 1.8 2. ]


In [8]:
# Incorporamos los valores reales
aprox[1,0] = w0
aprox[1,1] = ftyR(aprox[0,1])
aprox[1,2] = ftyR(aprox[0,2])

In [15]:
# Definimos el método de Adams - Moulton de tres pasos
def adamsM3(t, h, z, w0, w1, w2):
    adamsM3 = w0 + (h / 24) * (9 * fty(t + h, z) + 19 * fty(t, w0) - 5 * fty(t - h, w1) + fty(t - 2 * h, w2))
    adamsM3 = adamsM3 / 0.85
    return adamsM3

In [16]:
# Comenzamos el proceso iterativo
for i in range(3,n+1):
    aprox[1,i] = adamsM3(aprox[0,i-1], h, 0, aprox[1,i-1], aprox[1,i-2], aprox[1,i-3])

# Creamos el arreglo donde trabajaremos
resumen = np.empty((4,n+1))

# Asignamos los puntos donde trabajamos y los valores aproximados
resumen[0:2,:] = aprox.copy()

# Asignamos los valores reales
resumen[2,:] = ftyR(aprox[0,:])

# Determinamos el error de aproximación
resumen[3,:] = abs(resumen[2,:].copy() - aprox[1,:].copy())

In [17]:
# Imprimimos los resultados obtenidos
print('La aproximacion obtenida se encuentra dada por:')

# Titulos de la tabla
print(f'{punto:15}   {aproxima:15}  {real:15}  {error:15}')

for i in range(n+1):
    print('{0:15}   {1:15}   {2:15}   {3:15}'.format(round(resumen[0,i],8), round(resumen[1,i],8), round(resumen[2,i],8), round(resumen[3,i],8)))

La aproximacion obtenida se encuentra dada por:
Punto             Aproximacion     Real             Error Absoluto 
            0.0               2.0               2.0               0.0
            0.2        3.28201433        3.28201433               0.0
            0.4        5.34129823        5.34129823               0.0
            0.6        8.63541356          8.632304        0.00310956
            0.8       13.87813747       13.86849079        0.00964669
            1.0       22.18929666        22.1671683        0.02212836
            1.2        35.3191562       35.27416442        0.04499178
            1.4       55.99745012       55.91179902         0.0856511
            1.6       88.47351357       88.31710871        0.15640486
            1.8      139.35079492      139.07329089        0.27750404
            2.0      218.87467807      218.39260013        0.48207794
